### final Gold work

In [0]:
%sql
USE CATALOG ppe;
USE SCHEMA gold;


In [0]:
%sql
--Safety checks (fail fast)
SELECT COUNT(*) AS valid_cnt
FROM ppe.silver.silver_transactions
WHERE dq_status = 'VALID'

In [0]:
%sql
--Identify months impacted in this run
CREATE OR REPLACE TEMP VIEW silver_impacted_months AS
SELECT DISTINCT
    date_format(transaction_time, 'yyyy-MM') AS year_month
FROM ppe.silver.silver_transactions
WHERE dq_status = 'VALID';

In [0]:
%sql
-- Create Gold tables (one-time, safe to rerun)
CREATE TABLE IF NOT EXISTS gold_monthly_expenses (
    year_month STRING,
    category STRING,
    total_amount DOUBLE,
    transaction_count INT,
    load_ts TIMESTAMP
)
USING DELTA;


In [0]:
%sql
-- Daily Expenses
CREATE TABLE IF NOT EXISTS gold_daily_expenses (
    transaction_date DATE,
    year_month STRING,
    category STRING,
    total_amount DOUBLE,
    transaction_count INT,
    debit_amount DOUBLE,
    credit_amount DOUBLE,
    load_ts TIMESTAMP
)
USING DELTA;


In [0]:
%sql
-- Merchant Summary
CREATE TABLE IF NOT EXISTS gold_merchant_summary (
    year_month STRING,
    merchant STRING,
    category STRING,
    total_amount DOUBLE,
    transaction_count INT,
    load_ts TIMESTAMP
)
USING DELTA;


### Delete only impacted months(retention logic)

In [0]:
%sql
--Monthly table cleanup
DELETE FROM gold_monthly_expenses
WHERE year_month IN (SELECT year_month FROM silver_impacted_months);


In [0]:
%sql
--Daily table cleanup
DELETE FROM gold_daily_expenses
WHERE year_month IN (SELECT year_month FROM silver_impacted_months);


In [0]:
%sql 
--Merchant table cleanup
DELETE FROM gold_merchant_summary
WHERE year_month IN (SELECT year_month FROM silver_impacted_months);


### Insert refreshed aggregates

In [0]:
%sql
-- Insert into gold_monthly_expenses
INSERT INTO gold_monthly_expenses
SELECT
    date_format(transaction_time, 'yyyy-MM') AS year_month,
    category,
    SUM(amount) AS total_amount,
    COUNT(*) AS transaction_count,
    current_timestamp() AS load_ts
FROM ppe.silver.silver_transactions
WHERE dq_status = 'VALID'
GROUP BY date_format(transaction_time, 'yyyy-MM'), category;


In [0]:
%sql
-- Insert into gold_daily_expenses
INSERT INTO gold_daily_expenses
SELECT
    DATE(transaction_time) AS transaction_date,
    date_format(transaction_time, 'yyyy-MM') AS year_month,
    category,
    SUM(amount) AS total_amount,
    COUNT(*) AS transaction_count,
    SUM(CASE WHEN type = 'DEBIT' THEN amount ELSE 0 END) AS debit_amount,
    SUM(CASE WHEN type = 'CREDIT' THEN amount ELSE 0 END) AS credit_amount,
    current_timestamp() AS load_ts
FROM ppe.silver.silver_transactions
WHERE dq_status = 'VALID'
GROUP BY
    DATE(transaction_time),
    date_format(transaction_time, 'yyyy-MM'),
    category;


In [0]:
%sql
-- Insert into gold_merchant_summary
INSERT INTO gold_merchant_summary
SELECT
    date_format(transaction_time, 'yyyy-MM') AS year_month,
    merchant,
    category,
    SUM(amount) AS total_amount,
    COUNT(*) AS transaction_count,
    current_timestamp() AS load_ts
FROM ppe.silver.silver_transactions
WHERE dq_status = 'VALID'
GROUP BY
    date_format(transaction_time, 'yyyy-MM'),
    merchant,
    category;


In [0]:
%sql
-- Sanity checks
SELECT
  (SELECT SUM(amount)
   FROM ppe.silver.silver_transactions
   WHERE dq_status = 'VALID') AS silver_total,

  (SELECT SUM(total_amount)
   FROM gold_daily_expenses) AS gold_total;
